In [0]:
import requests
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType

headers = {
    'accept': 'application/json',
    'x-access-token-ws': 'd1bf4238656c2167c5997e4081b549eedd0a25622a18740d8ad1a8a31ee22b15'
}

# -------------------- EXTRAÇÃO ---------------------------------------
url = 'https://api.irancho.com.br/api/animal'
response = requests.get(url, headers=headers)
animals_list = response.json()
df_animal_pandas = pd.json_normalize(animals_list, sep='.')
print(f'animal {response.status_code}')

# ✅ CONVERTER TUDO PARA STRING NO PANDAS
df_animal_pandas = df_animal_pandas.astype(str)

# Criar schema com tudo como String
schema = StructType([
    StructField(col, StringType(), True) 
    for col in df_animal_pandas.columns
])

df_animal = spark.createDataFrame(df_animal_pandas, schema=schema)
df_animal.write.mode("overwrite").saveAsTable("animal_raw")

print("✅ Tabela criada com sucesso!")

In [0]:
display(df_animal)

Definições das tabelas 
